This is a filter that will act as a way to make sure that the users input is not harmful or dangerous. We can assum that here we want to filter the users response before it reaches the rest of the chatbot.

In [2]:
from transformers import pipeline

/opt/anaconda3/envs/xgb_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Create the gaurdrails

In [3]:
toxicity_classifier = pipeline(
    "text-classification", 
    model="unitary/toxic-bert", 
    top_k=None # Return scores for all categories
)

Device set to use mps:0


Create the safety function to be implemented into the chatbot

In [4]:
def check_safety(user_text, threshold=0.7):
    """
    Returns (True, Reason) if safe.
    Returns (False, Reason) if unsafe.
    """
    results = toxicity_classifier(user_text)
    
    # The model returns a list of lists. We grab the first result.
    scores = results[0]
    
    # Check if any category exceeds the threshold
    for category in scores:
        if category['score'] > threshold:
            return False, f"Blocked due to {category['label']} ({round(category['score'], 2)})"
    
    return True, "Safe"

Test the filter

In [5]:
test_messages = [
    "Where is my refund? I am frustrated.",  # Negative but Safe
    "You are a stupid bot and I hate you.",   # Insult
    "I'm going to destroy your office."       # Threat
]

In [6]:
for msg in test_messages:
    is_safe, reason = check_safety(msg)
    status = "PASS" if is_safe else "BLOCK"
    print(f"[{status}] Message: '{msg}' -> {reason}")

[PASS] Message: 'Where is my refund? I am frustrated.' -> Safe
[BLOCK] Message: 'You are a stupid bot and I hate you.' -> Blocked due to toxic (0.99)
[BLOCK] Message: 'I'm going to destroy your office.' -> Blocked due to toxic (0.82)
